# Code pour nettoyer et manipuler les csv d'export des corpus de Thérèse Bonney et de l'Exposition Universelle de 1937 en vue du scraping 🧹

*Les exports en csv présentaient des irrégularités et des coquilles empêchant l'automatisation.*

## Environnement 🌐

### Import des librairies 📖

In [110]:
import io
import re
import pandas as pd
from contextlib import redirect_stderr

## Fonctions 🤖

### Fonction pour le csv "Expo_1937_photos.csv" 🏗️

In [ ]:
def cleaning_expo(input_file_expo):
    remove_first_line_csv(input_file_expo)
    bad_lines, parsing_error = detect_bad_lines_expo(input_file_expo)
    try:
        df = pd.read_csv(input_file_expo)
    except Exception as e:
        print("Erreur de parsing :", e)
    clean_url(input_file_expo)
    clean_quotes_and_expressions(input_file_expo)

### Suppression première ligne du csv 🥇

*La première ligne du csv comprend une seule colonne avec "rapport_33396" ce qui empêche pandas de lire correctement l'en-tête*

**La fonction doit être commentée si on refait une passe**

In [112]:
def remove_first_line_csv(filepath):
    """Supprime la première ligne d'un fichier CSV (en place)."""
    with open(filepath, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    with open(filepath, 'w', encoding='utf-8') as f:
        f.writelines(lines[1:])

### Détecter les lignes fautives d'un csv 📏

+ Utilisation de la librairie io et de redirect_stderr pour capturer les lignes fautives du csv ne comportant pas le bon nombre de colonnes

*Le besoin s'est fait ressentir de passer par une phase préalable de troubleshooting face à l'impossibilité de lire le csv avec un simple pd.read_csv*

Excepté une petite modification dû à un changement dans la syntaxe de pandas dans la version 1.3.0 (concernant le paramètre "on_bad_lines").

Les crédits vont à :

© vlogize https://www.youtube.com/watch?v=1fdePybJ0yQ
CC BY-SA 4.0

Based on a Stack Overflow exchange https://stackoverflow.com/users/6238676/prish
Also under CC BY-SA 4.0

In [113]:
def detect_bad_lines_expo(input_file_expo, sep=';', encoding='utf-8'):
    """
    Détecte les lignes fautives (mauvais nombre de colonnes) dans un CSV.
    Retourne la liste des numéros de lignes problématiques.
    """
    # Prepare capture warnings
    f = io.StringIO()
    parsing_error = None
    with redirect_stderr(f):
        try:
            pd.read_csv(input_file_expo, sep=sep, encoding=encoding, on_bad_lines='warn')
        except Exception as e:
            parsing_error = str(e)
    # Extract warning messages
    warning_str = f.getvalue()

    # Using Regex to capture line numbers
    regex = re.compile(r'line ([0-9]+)')
    line_numbers = regex.findall(warning_str)
    
    return line_numbers, parsing_error

### Suppression des espaces qui se sont glissés dans l'export 🌌

*Les adresses des sites des bibliothèques spécialisées de Paris comportaient un espace entre la fin de l'adresse et le ".fr" empêchant l'automatisation*

Codé avec Github Copilot

In [114]:
def clean_url(input_file_expo):
    """
    Nettoie les URLs du fichier CSV en supprimant l'espace dans 'paris .fr'.
    Modifie le fichier en place.
    """
    import csv

    def clean_url(url):
        return url.replace("paris .fr", "paris.fr")

    # Lire tout le contenu et nettoyer
    with open(input_file_expo, mode='r', encoding='utf-8') as infile:
        rows = []
        reader = csv.reader(infile)
        for row in reader:
            cleaned_row = [clean_url(cell) if "https://bibliotheques-specialisees.paris" in cell else cell for cell in row]
            rows.append(cleaned_row)

    # Réécrire le fichier nettoyé
    with open(input_file_expo, mode='w', encoding='utf-8', newline='') as outfile:
        writer = csv.writer(outfile)
        writer.writerows(rows)

### Supression des coats et des expressions problématiques 🧥

Codé avec Github Copilot

In [115]:
def clean_quotes_and_expressions(input_file_expo):
    """
    Supprime les guillemets et expressions problématiques dans le fichier CSV.
    Modifie le fichier en place.
    """
    with open(input_file_expo, "r", encoding="utf-8") as file:
        content = file.read()

    # Suppression des guillemets et expressions spécifiques via des RegEx
    content = re.sub(r'\"\"\"', '', content)
    content = re.sub(r'\"\"', '', content)
    content = re.sub(r'\"', '', content)
    content = re.sub(r'\"https', 'https', content)
    content = re.sub(r'\", Thérèse', ', Thérèse', content)
    content = re.sub(r'\", Paris', ', Paris', content)
    content = re.sub(r',\" inauguration', ' inauguration', content)
    content = re.sub(r',\" la brasserie', ' la brasserie', content)
    content = re.sub(r', avec Marcel Maillard', ' avec Marcel Maillard', content)
    content = re.sub(r', des Gobelins', ' des Gobelins', content)
    content = re.sub(r', architectes du', ' architectes du', content)
    content = re.sub(r'BHVP, *', 'BHVP;', content)
    content = re.sub(r'BHVP;.*', ' BHVP;', content)
    content = re.sub(r'\"La Décoratrice\"\",\"', '\"La Décoratrice\"', content)
    content = re.sub(r' , Paris 1937', ' Paris 1937', content)
    content = re.sub(r',', '', content)

    with open(input_file_expo, "w", encoding="utf-8") as file:
        file.write(content)

    print("Les guillemets et expressions problématiques ont été supprimés avec succès.")

### Fonction pour le csv "Corpus_Therese_Bonney_BnF.csv" 🏗️

In [116]:
def cleaning_bonney(input_file_bonney):
    bad_lines, parsing_error = detect_bad_lines_bonney(input_file_bonney)
    try:
        df = pd.read_csv(input_file_bonney)
    except Exception as e:
        print("Erreur de parsing :", e)
    clean_quotes_and_expressions_bonney(input_file_bonney)

In [117]:
def detect_bad_lines_bonney(input_file_bonney, sep=';', encoding='utf-8'):
    """
    Détecte les lignes fautives (mauvais nombre de colonnes) dans un CSV.
    Retourne la liste des numéros de lignes problématiques.
    """
    # Prepare capture warnings
    f = io.StringIO()
    parsing_error = None
    with redirect_stderr(f):
        try:
            pd.read_csv(input_file_bonney, sep=sep, encoding=encoding, on_bad_lines='warn')
        except Exception as e:
            parsing_error = str(e)
    # Extract warning messages
    warning_str = f.getvalue()

    # Using Regex to capture line numbers
    regex = re.compile(r'line ([0-9]+)')
    line_numbers = regex.findall(warning_str)
    
    return line_numbers, parsing_error

In [118]:
def clean_quotes_and_expressions_bonney(input_file_expo):
    """
    Supprime les guillemets et expressions problématiques dans le fichier CSV.
    Modifie le fichier en place.
    """
    with open(input_file_expo, "r", encoding="utf-8") as file:
        content = file.read()

    # Suppression des guillemets et expressions spécifiques
    content = re.sub(r'\"\"\"', '', content)
    content = re.sub(r'\"\"', '', content)
    content = re.sub(r'\"', '', content)
    content = re.sub(r',', '', content)

    with open(input_file_expo, "w", encoding="utf-8") as file:
        file.write(content)

    print("Les guillemets et expressions problématiques ont été supprimés avec succès.")

## Processing ⚙️

In [ ]:
input_file_expo = ""
input_file_bonney = ""

### Lancement du traitement pour le csv Exposition Universelle 1937 🌍

In [ ]:
cleaning_expo(input_file_expo)

**Si la fonction renvoie une "Erreur de parsing : Error tokenizing data. C error: Expected x fields in line y, saw z"**

Relancer la fonction de cleaning **en commentant la fonction de suppression de la première ligne**

Si le problème persiste :

Alors ouvrir le csv dans Libreoffice, enregistrer et le refermer. Cela règle le problème en réecrivant un nombre cohérent de colonnes. 

### Lancement du traitement pour le csv Thérèse Bonney 📷

In [ ]:
cleaning_bonney(input_file_bonney)

**Si la fonction renvoie une "Erreur de parsing : Error tokenizing data. C error: Expected x fields in line y, saw z"**

Relancer la fonction de cleaning 

Si le problème persiste :

Alors ouvrir le csv dans Libreoffice, enregistrer et le refermer. Cela règle le problème en réecrivant un nombre cohérent de colonnes. 